### Langchain ReAct + SerpAPI/Google CSE API

In [2]:
import os
from dotenv import load_dotenv
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import OpenAI
from langchain_community.utilities import SerpAPIWrapper

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Tool 1: 用 SerpAPI 搜尋
serpapi_search = SerpAPIWrapper()
serpapi_tool = Tool(
    name="SerpAPI Search",
    func=serpapi_search.run,
    description="使用 SerpAPI 查詢 Google 搜尋結果"
)

# Tool 2: 用 Google CSE API 搜尋
def google_cse_search(query: str, **kwargs):
    try:
        from googleapiclient.discovery import build
        service = build(
            serviceName="customsearch",
            version="v1",
            developerKey=GOOGLE_API_KEY,
            static_discovery=False
        )
        res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, **kwargs).execute()
        items = res.get("items", [])
        snippets = [item.get("snippet", "") for item in items]
        return "\n".join(snippets[:3]) if snippets else "none"
    except Exception as e:
        return f"Google CSE error: {str(e)}"

cse_tool = Tool(
    name="Google CSE Search",
    func=google_cse_search,
    description="使用 Google 自訂搜尋 API"
)

llm = OpenAI(temperature=0)

# 整合兩個搜尋工具給 Agent 用
tools = [serpapi_tool, cse_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 測試
article_content = """現在(2025/3/27)的特斯拉股價是330"""
response = agent.run(f"Please go to external websites to help me look up the related information. Article: \"{article_content}\". Based on your analysis, and all the information you search. Does this article contain any misleading information or mismatched relationships? Show your analytic process and If this article is more likely to contain misleading information, return 1; otherwise, return 0. The answer (Arabic numerals) is:")
print("📌 Answer：", response)




> Entering new AgentExecutor chain...
 I should use a search engine to look up the current Tesla stock price and compare it to the one mentioned in the article.
Action: SerpAPI Search
Action Input: "Tesla stock price"
Observation: {'type': 'finance_results', 'title': 'Tesla Inc', 'exchange': 'NASDAQ', 'stock': 'TSLA', 'currency': 'USD', 'price': 295.14}
Thought: The current Tesla stock price is lower than the one mentioned in the article, so it may contain misleading information.
Action: SerpAPI Search
Action Input: "Tesla stock price 2025/3/27"
Observation: ['Tesla, Inc. (TSLA) ; May 29, 2025, 365.29, 367.71 ; May 28, 2025, 364.84, 365.00 ; May 27, 2025, 347.35, 363.79 ; May 23, 2025, 337.92, 343.18 ...', 'Tesla, Inc. (TSLA) ; May 29, 2025, 365.29, 367.71 ; May 28, 2025, 364.84, 365.00 ; May 27, 2025, 347.35, 363.79 ; May 23, 2025, 337.92, 343.18 ...', "Join us as we analyze Tesla's key price levels and potential breakout zones in today's volatile market—can bulls gain momentum?", "